**Import Pytroch Libraries and Set Device**

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import json
import math 
import time
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torchtext.data.metrics import bleu_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

**Load data in tab space format: Input Sentence - Output Sentence **

In [2]:
# read file
with open('../input/squadpreprocessed/data/dev_eval.json', 'r') as myfile:
    input_data=myfile.read()

# parse file
obj = json.loads(input_data)

data = []
# show values
for i in range(1, len(obj)+1):
    line = obj[str(i)]['context'] + "\t" + obj[str(i)]['question']
    data.append(line)

print(data[0:3])


KeyboardInterrupt: 

Used code from [pytorch seq2seq_translation_tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) to preprocess data. Have replaced lang class to be vocab (as we are dealing with Question Generation) and removed filtering by length or content.

In [ ]:
PAD_token = 0
SOS_token = 1
EOS_token = 2


class Vocab:
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0:"PAD", 1: "SOS", 2: "EOS"}
        self.n_words = 3  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


def readNormalizedPairs():
    print("Reading lines...")

    # Read the file and split into lines
    lines = data[0:6000]

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines[0:5700]]
    pairs_test = [[normalizeString(s) for s in l.split('\t')] for l in lines[5700:6000]]
    
    return pairs, pairs_test


def prepareData():
    vocab = Vocab()
    pairs, pairs_test = readNormalizedPairs()
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs+pairs_test:
        vocab.addSentence(pair[0])
        vocab.addSentence(pair[1])
    print("Counted words:")
    print(vocab.n_words)
    return vocab, pairs, pairs_test


vocab, pairs, pairs_test = prepareData()
print(random.choice(pairs))

**RNN Encoder** - Reference - [pytorch seq2seq_translation_tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

**RNN Decoder** - Reference - [pytorch seq2seq_translation_tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Functions: Preparing Training Data - Reference - [pytorch seq2seq_translation_tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

In [ ]:
def indexesFromSentence(vocab, sentence):
    return [vocab.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(vocab, sentence):
    indexes = indexesFromSentence(vocab, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(vocab, pair[0])
    target_tensor = tensorFromSentence(vocab, pair[1])
    return (input_tensor, target_tensor)

# Helper Functions for printing time elapse

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

**Training the Model** - [pytorch seq2seq_translation_tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

In [ ]:
teacher_forcing_ratio = 0.5
MAX_LENGTH = 3000

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

Training Iterations - reference: [pytorch seq2seq_translation_tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

In [ ]:
#Helper Function to show plots

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

**Evaluation** - reference: [pytorch seq2seq_translation_tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(vocab, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(vocab.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

def evaluateRandomly(encoder, decoder, pairs, n=6):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

**Training and Evaluating** - reference: [pytorch seq2seq_translation_tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(vocab.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, vocab.n_words).to(device)

trainIters(encoder1, decoder1, 40000, print_every=1000)

**Evaluate the Model randomly**

In [ ]:
# Evaluate Train Set 
evaluateRandomly(encoder1, decoder1, pairs)

# Evaluate Test Set
evaluateRandomly(encoder1, decoder1, pairs_test)

**Evaluate BLEU Score** - First create an iterable of reference questions for each context. And then apply BLEU score to them.

In [ ]:
def evaluate_pair(encoder, decoder, context):
    output_words = evaluate(encoder, decoder, context)
    output_sentence = ' '.join(output_words)
    return output_sentence

def FindCandidatesAndReferencesForBLEU(pairs):
    dic  = {}
    candidates = {}
    for c, q in pairs:
        if c in dic.keys():
            dic[c].append(q.split())
        else:
            dic[c] = [q.split()]
        candidates[c] = evaluate_pair(encoder1, decoder1, c).split()
    return list(candidates.values()), list(dic.values())

# Estimate BLEU for set

def estimateBLEU(pairs, set_name):
    for i in range(1,5):
        candidate_corpus, references_corpus = FindCandidatesAndReferencesForBLEU(pairs)
        bleu_test = bleu_score(candidate_corpus, references_corpus, max_n=i, weights=[1./i]*i)
        print("BLEU-" + str(i) + " on " + set_name + " :" + str(bleu_test))
    return

# Estimate BLEU for test set
estimateBLEU(pairs_test, "Test")

# Estimate BLEU for train set
estimateBLEU(pairs, "Train")
